In [3]:
import pandas as pd
import numpy as np

In [73]:
df_train = pd.read_csv('train.csv',
                       sep=',', encoding='utf8')

In [150]:
#все таки нужно объединить тест и трейн, чтобы были одинаковые поля
#обучить модель на тесте , а потом предсказать
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16476 entries, 0 to 16475
Data columns (total 21 columns):
_id               16476 non-null object
age               16476 non-null int64
job               16476 non-null object
marital           16476 non-null object
education         16476 non-null object
default           16476 non-null object
housing           16476 non-null object
loan              16476 non-null object
contact           16476 non-null object
month             16476 non-null object
day_of_week       16476 non-null object
duration          16476 non-null int64
campaign          16476 non-null int64
pdays             16476 non-null int64
previous          16476 non-null int64
poutcome          16476 non-null object
emp.var.rate      16476 non-null float64
cons.price.idx    16476 non-null float64
cons.conf.idx     16476 non-null float64
euribor3m         16476 non-null float64
nr.employed       16476 non-null float64
dtypes: float64(5), int64(5), object(11)
memory usa

In [ ]:
_id - идентификатор человека
age - возраст
job - тип работы ("admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")
marital - семейное положение ("divorced" (разведенный и овдовевший),"married","single","unknown")
education - уровень образования ("basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
default - есть ли дефолт по кредиту ("no","yes","unknown")
housing - есть ли кредит на жилье ("no","yes","unknown")
loan - есть ли персональный кредит ("no","yes","unknown")
contact - контактный тип связи ("cellular","telephone")
month - последний контактный месяц года ("jan", "feb", "mar", ..., "nov", "dec")
day_of_week - последний контактный день недели ("mon","tue","wed","thu","fri")
duration - длительность последнего контакта в секундах. (Важное замечание: этот атрибут сильно влияет на таргет (например, если длительность - 0 сек., то таргет - 0). Тем не менее, продолжительность не известна до выполнения вызова.
campaign - количество контактов, выполненных для этого клиента в рамках текущей маркетинговой кампании (включая последний контакт)
pdays - количество дней прошедших последнего контакта с клиентом с предыдущей маркетинговой кампании (999 означает, что ранее с клиентом не контактировали)
previous - количество контактов, выполненных до текущей маркетинговой кампании для этого клиента
poutcome - результат предыдущей маркетинговой кампании ("failure","nonexistent","success")
# ниже приведены показатели окружающей среды и банка
emp.var.rate - изменение уровня занятости (квартальный показатель)
cons.price.idx - индекс потребительских цен (ежемесячный показатель)
cons.conf.idx - индекс доверия потребителя (ежемесячный показатель)
euribor3m - курс Euribor (ежедневный показатель)
nr.employed - количество сотрудников (квартальный показатель)
target - целевая переменная (таргет) - согласился ли клиент на вклад (депозит)

In [160]:
df_train.shape

(24712, 22)

In [158]:
df_test = pd.read_csv('test.csv',
                       sep=',', encoding='utf8')

In [159]:
df_test.shape

(16476, 21)

In [129]:
def preproc_df(df_input):
    df_loc = df_input.copy()
    #может для пенсии сделать отдельный параметр?
    def proc_job(i):
        if (i != "unknown") & (i != "unemployed"):
            i = "employed"
        return i
    df_loc['job'] = df_loc['job'].map(proc_job)
    
    def proc_education(i):
        if i in ["basic.4y","basic.6y","basic.9y"]:
            i = "school"
        elif i in ["high.school","professional.course","university.degree"]:
            i = "high"
        return i
    df_loc['education'] = df_loc['education'].map(proc_education)
    df_loc = df_loc.drop(['month','day_of_week','_id'], axis=1)
    
    df_loc = pd.get_dummies(df_loc, columns=['job','marital', 'education','default','housing','loan','contact','poutcome'])
    
    return df_loc

In [140]:
df_train1 = ''
df_train1 = df_train.pipe(preproc_df)

In [141]:
X = df_train1.drop(['target'], axis=1)
y = df_train1['target']

In [164]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
tree = DecisionTreeClassifier()
tree.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [165]:
tree.predict(X)

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [156]:
list1=cross_val_score(
    tree,
    X,
    y,
    scoring='roc_auc',
    cv=15
)
print(list1)
print(list1.mean())

[0.76516556 0.69942486 0.74746996 0.73872145 0.76692335 0.71887825
 0.7224968  0.7134541  0.75074651 0.70983555 0.74207348 0.75815283
 0.72055053 0.73474851 0.71210019]
0.7333827938898696


In [166]:
predic_proba_tree = tree.predict_proba(df_test)

ValueError: could not convert string to float: 'nonexistent'

In [147]:
df_test1 = ''
df_test1 = df_test.pipe(preproc_df)

KeyError: "['target'] not found in axis"